# $(\mathrm{H}_2\mathrm{O})_6$ cost estimate

In [1]:
import numpy as np
import openfermion
import pyscf
from pyscf import gto

/home/mitarai/anaconda/envs/qulacs/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
xyz_file = "TIP4P-6.xyz"

Get energy gap of a single molecule

In [3]:

atom = [('O', [-0.3511375701099324, -2.220327851906733, 1.4480207843187745]), ('H', [-0.24292674995680064, -1.697798925312798, 3.1763661875721043]), ('H', [-0.11012983703262759, -0.699680546250837, 0.4985624750191387]),]
basis = 'sto3g'
mol = gto.M(atom=atom,
            basis=basis,
            unit="Bohr")
rhf = mol.RHF.run()
cisolver = pyscf.fci.FCI(rhf)
cisolver.nroots = 5
e0, e1, e2, e3, e4 = cisolver.kernel()[0]
Delta = e1-e0
p = cisolver.ci[0][0,0]**2

converged SCF energy = -74.962928260405


Get the Hamiltonian

In [4]:
from local_hamiltonian import get_localized_hamiltonian, get_localized_integrals
basis = 'sto3g'
method = 'lowdin'
mol = gto.M(atom="TIP4P-6.xyz",
            basis=basis)
rhf = mol.RHF.run()
e_rhf = rhf.e_tot
of_hamiltonian = get_localized_hamiltonian(mol, method)
constant, one_body_int, two_body_int = get_localized_integrals(mol, method)
# of_hamiltonian.constant = 0  # constant term value

converged SCF energy = -449.856772094009
converged SCF energy = -449.856772094008
converged SCF energy = -449.856772094009


Split the Hamiltonian

In [5]:
groups = [list(range(14*i, 14*i+14)) for i in range(6)]
original_qubit_hamiltonian = openfermion.transforms.jordan_wigner(of_hamiltonian)
constant_in_original_qubit_hamiltonian = original_qubit_hamiltonian.terms[()]
original_qubit_hamiltonian -= constant_in_original_qubit_hamiltonian
# Note that original_qubit_hamiltonian should not have constant hereafter. 

In [6]:
from utils import local_group_qubit_operator, get_coulomb_like_interaction_group
hamiltonian_list = []
for i in range(6):
    hamiltonian_list.append(local_group_qubit_operator(original_qubit_hamiltonian, groups[i]))
for i in range(6):
    hamiltonian_interaction = original_qubit_hamiltonian - hamiltonian_list[i]

Get the pauli coefficients

In [7]:
def coef2array(h):
    _array = np.array([h.terms[key] for key in h.terms])
    return _array[np.where(np.abs(_array)>1e-11)]
original_coef = coef2array(original_qubit_hamiltonian)
grouped_coef_list = []
for i in range(6):
    grouped_coef_list.append(coef2array(hamiltonian_list[i]))
interaction_coef = coef2array(hamiltonian_interaction)

Calculate norms

In [8]:
h1norm = abs(e_rhf-constant_in_original_qubit_hamiltonian)
for i in range(6):
    h1norm += np.sum(np.abs(grouped_coef_list[i]))
v1norm = np.sum(np.abs(interaction_coef))
v23norm = np.sum(np.abs(interaction_coef)**(2/3))**(3/2)
lvlh = len(interaction_coef)/(sum([len(grouped_coef_list[i]) for i in range(6)]))
print(h1norm)
print(v1norm)
print(v23norm)
print(lvlh)

715.7711319290972
610.3147600518222
70281.74518678922
223.84920634920636


# Generate values in the paper

In [9]:
from cost_estimate import get_cost_for_first_order_pert, get_cost_for_second_order_pert
delta0 = 0.3e-3
delta1 = 0.3e-3
delta2 = 0.3e-3
r_scale = 1/6
epsilon_scale = 1/6
n_subsystems = 6
print(get_cost_for_first_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))
print(get_cost_for_second_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))

epsilon:  2.4885699817798904e-08
n_filter:  (351282.31419988244+0j)
kappa:  0.0005562915466404593
M1:  1068543.261988356
r:  4.096246991941868e-09
x_th: 4.1912838701871164e-07
(91327677775044.27+0j)
r:  2.6744592805578082e-12
epsilon_filter:  1.6247992604405086e-11
epsilon_ptb:  5.764845692084068e-08
kappa:  0.0005562915466404593
w:  0.0005562915466404593
w0:  4.1912838701871164e-07
M2:  1257380581249660.5
n_filter:  (493027.6391437042+0j)
n_ptb:  (2291113.3277967963+0j)
(2.090052699786265e+23+0j)
